<a href="https://colab.research.google.com/github/tomonari-masada/course2023-stats1/blob/main/03_text_retrieval_with_multinomial_distributions_(answer).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 課題20231017の答案例

## 対数尤度を求めるヘルパ関数における工夫

* 計算の高速化をしてみる。
  * 確率はスムージングをしてある。
  * だから、**推定された単語確率はゼロには絶対にならない**。
  * これを前提として、クエリの尤度の計算を高速化することを試みる。

以下は、授業で使ったnotebookからの抜粋。

---
* クエリの尤度を、各文書について求めた単語確率を使って計算する。
  * $n_{q,w}$: クエリ$q$における単語$w$の出現頻度
  * このとき、文書$d$の単語確率を使ったクエリ$q$の対数尤度は、以下の通り。
$$\begin{align}
L_q(d) = \sum_w n_{q,w} \log \phi_{d,w}
\end{align}$$
  * 上の式で、規格化定数の部分は省略している。（ランキングに関係しないため。）
* このように計算されたクエリの尤度によって、検索対象の文書をソートする。
  * $L_q(d)$が大きい順に、文書を検索結果として表示する。
---

* 上の説明にある$L_q(d)$の式の計算を、高速化する。
* 今回は、GPUも使って、高速化する。

**ランタイムのタイプをGPUにしておく**

## 準備

In [10]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

## データセット

In [11]:
train_corpus, train_labels = fetch_20newsgroups(subset="train", return_X_y=True)
test_corpus, test_labels = fetch_20newsgroups(subset="test", return_X_y=True)
print(f"training size: {len(train_corpus)}\ntest size: {len(test_corpus)}")

training size: 11314
test size: 7532


## 単語の出現回数を数える

In [12]:
vectorizer = CountVectorizer(min_df=10, stop_words="english")
X_train = vectorizer.fit_transform(train_corpus).toarray()
X_test = vectorizer.transform(test_corpus).toarray()
vocabulary = vectorizer.get_feature_names_out()
print(f"vocabulary size: {len(vocabulary)}")

vocabulary size: 15291


## スムージング付きで単語の確率を計算する

In [13]:
# ディリクレ事前分布のパラメータ
beta = 0.01

X_train = X_train + beta
X_train_probs = X_train / X_train.sum(axis=1).reshape(-1, 1)

## クエリの対数尤度を計算するヘルパ関数
* PyTorchを使って、GPU上で計算する。

In [14]:
import torch

def log_likelihood(x_test, x_train_prob):
  return torch.matmul(
      torch.tensor(x_test, dtype=torch.float32, device="cuda"),
      torch.log(torch.tensor(x_train_prob, dtype=torch.float32, device="cuda")).t()
  ).cpu().numpy()

## 検索の実行

* 全てのtest文書について、その尤度を最大にするtraining文書を求める。

In [15]:
scores = log_likelihood(X_test, X_train_probs)

In [16]:
sorted_train_indices = (- scores).argsort(-1)

In [17]:
top_ranked_train_docs = sorted_train_indices[:,0].reshape(-1)
print(top_ranked_train_docs)

[9048 4114 7539 ... 2018 2965 7340]


* P@1はprecision at oneの略。
  * https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)#Precision_at_k

In [18]:
print(f"P@1={(test_labels == train_labels[top_ranked_train_docs]).sum()/len(test_labels):.3f}")

P@1=0.703
